## Simulation in the Inventory Model ( Julia )

This code simulates the inventory of a firm many times.

It generates a matrix with `num_paths` rows and `sim_length` columns.

One simulation of inventory dynamics corresponds to one row.  

The question is: how well does naive parallelization work?

In [1]:
using Distributions

Threads.nthreads()

4

In [2]:
mutable struct SP
    s::Float64
    S::Float64
    p::Float64
end


function SP()
    
    s = 10.0
    S = 100.0
    p = 0.4
    SP(s, S, p)   
end

SP

## Sequential:

In [3]:
function sim_paths(sp::SP;initial_x=50.0, num_paths=100000,sim_length=400)
    
    X = Array{Int32}(num_paths,sim_length)
    X[:, 1] = initial_x
    
    # For each row
    for i = 1:num_paths
        
        dvals = rand(Geometric(sp.p),sim_length)
        for t = 1:sim_length-1
            x, d = X[i, t], dvals[t]
            if x <= sp.s
                y = max(sp.S - d, 0)
            else
                y = max(x - d, 0)
            end
            
            X[i, t+1] = y
            
            end
        end
       return X
    
end

sim_paths (generic function with 1 method)

In [4]:
sp = SP();

compilation time:

In [5]:
@time X = sim_paths(sp);

  5.165953 seconds (40.49 M allocations: 1.062 GiB, 8.72% gc time)


running time:

In [6]:
@time X = sim_paths(sp)

  4.235954 seconds (40.46 M allocations: 1.060 GiB, 6.61% gc time)


100000×400 Array{Int32,2}:
 50  49  48  47  47  46  43  42  41  …  14  13  11   9  100  92  90  89  89
 50  49  47  46  43  42  42  42  42     97  97  97  96   94  93  90  88  86
 50  50  48  48  44  43  42  42  41     39  36  35  34   30  30  30  28  28
 50  50  50  49  46  46  41  41  40     33  31  31  28   27  25  24  24  24
 50  46  44  44  44  43  43  42  42     25  25  23  21   21  20  19  19  19
 50  48  47  45  42  39  37  36  36  …  84  84  83  83   82  80  80  75  73
 50  41  40  38  38  38  38  36  36     12   9  96  96   95  95  93  91  89
 50  48  44  44  41  40  40  40  37     34  34  34  32   31  29  20  17  16
 50  50  49  48  48  40  38  33  33     96  96  91  89   84  84  83  82  81
 50  50  50  50  50  50  44  42  36     97  97  96  94   93  92  92  92  91
 50  49  49  46  44  43  43  43  41  …  74  73  70  69   68  64  63  63  62
 50  50  49  48  48  47  45  44  43     65  65  63  63   60  58  58  57  57
 50  47  47  45  43  39  33  33  32     21  19  16  10  100  

## Parallel:

In [7]:
function sim_paths(sp::SP;initial_x=50.0, 
                                num_paths=100000, sim_length=400)
    
    X = SharedArray{Int32}(num_paths,sim_length)
    X[:, 1] = initial_x
    
    # For each row
    Threads.@threads for i=1:num_paths
        dvals = rand(Geometric(sp.p),sim_length)
        
        for t=1:sim_length-1
            x, d = X[i, t], dvals[t]
            
            if x <= sp.s
                y = max(sp.S - d, 0)
            else
                y = max(x - d, 0)
            end
        
            X[i, t+1] = y
            
        end
    end
    
    return X  
end

sim_paths (generic function with 1 method)

compilation time:

In [8]:
@time X = sim_paths(sp);

  6.034088 seconds (30.64 M allocations: 735.131 MiB, 2.89% gc time)


running time:

In [9]:
@time X = sim_paths(sp)

  2.916295 seconds (28.52 M allocations: 654.490 MiB, 10.18% gc time)


100000×400 SharedArray{Int32,2}:
 50  50  47  46  40  40  38  38  38  …  36  33  33  33  31  30  30  29   29
 50  48  48  46  41  37  37  37  33     50  46  45  39  38  36  33  33   33
 50  47  46  46  40  34  30  30  26     55  55  55  54  54  51  46  45   44
 50  49  48  47  47  47  47  47  44     85  84  83  83  78  76  76  76   72
 50  50  46  46  43  43  43  43  40     66  66  65  64  64  64  64  64   63
 50  45  45  45  45  45  41  41  41  …  54  52  52  47  46  41  39  39   39
 50  50  50  43  42  41  35  35  35     82  81  81  77  77  76  76  70   70
 50  50  45  41  41  39  39  39  38     45  45  45  42  42  42  42  37   33
 50  45  45  45  44  44  43  41  39     58  58  58  56  55  55  55  53   53
 50  50  49  48  46  42  41  36  31     17  16  16  16  16  13  12  12   11
 50  49  47  46  45  45  43  42  42  …  39  39  39  37  37  33  32  32   30
 50  50  49  43  40  40  39  37  37     53  53  53  50  48  46  46  44   41
 50  49  42  36  36  36  36  36  34     54  54  46  44 